<a href="https://colab.research.google.com/github/romellfudi/medium/blob/main/Function_calling_with_OpenAI_API.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

https://platform.openai.com/docs/guides/gpt/function-calling

In [ ]:
#@title Installing python dependencies
%%capture
!pip install python-dotenv langchain openai tiktoken faiss-cpu  spotipy -q

In [ ]:
#@title To Environment keys { display-mode: "form" }
#@markdown Add your OPEN-API-KEY from [openai](https://openai.com/blog/openai-api)
#@markdown and add your spotify dev credentials from [SpotifyDev](https://developer.spotify.com/dashboard)</br>
#@markdown By default we use a developer-api-key

openai_key = ''  #@param {type: "string"}
spotipy_client_key = ''  #@param {type: "string"}
spotipy_secret_key = ''  #@param {type: "string"}

with open(".env", "w") as f:
    f.write(f"OPENAI_API_KEY={openai_key}\n")
    f.write(f"SPOTIPY_CLIENT_ID={spotipy_client_key}\n")
    f.write(f"SPOTIPY_CLIENT_SECRET={spotipy_secret_key}\n")

print("Overwriting .env")

from dotenv import load_dotenv
_=load_dotenv()

Overwriting .env


In [ ]:
import spotipy
import os
import openai
from spotipy.oauth2 import SpotifyClientCredentials
# Set up your Spotify API credentials
client_credentials_manager = SpotifyClientCredentials()
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

In [ ]:
import json
import pprint

def list_latest_episode_of_spotify_show(show_name, limit=40):
    """Get the latest episodes of a determine show or episode in a given name"""
    result_fetch = sp.search(show_name, limit=limit, offset=0, type='episode',market='ES')
    final_list = []
    for item in result_fetch['episodes']['items']:
        if show_name.lower() in item['description'].lower():
            final_list.append({'release_date':item['release_date'],'episode_name':item['name'],'url':item['external_urls']['spotify']})
    final_list = sorted(final_list, key=lambda x: x['release_date'], reverse=True)
    return json.dumps(final_list[:limit])

In [ ]:
result_fetch = list_latest_episode_of_spotify_show('Globant',3)
result_fetch

'[{"release_date": "2023-06-26", "episode_name": "Qu\\u00e9 es el Learning Match - Santos Videla (Globant)", "url": "https://open.spotify.com/episode/568pXcS72Cg9R8RlLsQCzP"}, {"release_date": "2023-06-20", "episode_name": "Argentinian tech giant Globant plans to double down on Europe: A chat with Fernando Matzkin, who\'s leading the charge", "url": "https://open.spotify.com/episode/2e7tBHgZhlsnekzoCf1xus"}, {"release_date": "2023-06-13", "episode_name": "Entrevista a Andrey Luj\\u00e1n, Managing Director de Globant en Chile", "url": "https://open.spotify.com/episode/1lYESmQLnB4PABIqH4xJlO"}]'

In [ ]:
functions = [
        {
            "name": "list_latest_episode_of_spotify_show",
            "description": "Get the latest episodes  from Spotify shows or episodes in a given name",
            "parameters": {
                "type": "object",
                "properties": {
                    "show_name": {
                        "type": "string",
                        "description": "The name of the show or episode, e.g., Apple, Globant, Twitter, Amazon",
                    },
                    "limit": {
                        "type": "integer",
                        "description": "The maximum number of items to fetch",
                    },

                },
                "required": ["show_name"],
            },
        }
    ]

In [ ]:
messages = [{'role': 'user', 'content': 'Hello, I need your help to find postcasts in my Spotify'}]
user_message="What are the latest ten episodes of Globant?"
messages.append({"role": "user", "content": user_message})
completion= openai.ChatCompletion.create(
    model="gpt-3.5-turbo",
    messages=messages,
    functions=functions
)

In [ ]:
response_function_calling=completion.choices[0].message
response_function_calling

<OpenAIObject at 0x7ff57ea42890> JSON: {
  "role": "assistant",
  "content": null,
  "function_call": {
    "name": "list_latest_episode_of_spotify_show",
    "arguments": "{\n  \"show_name\": \"Globant\",\n  \"limit\": 10\n}"
  }
}

In [ ]:
function_name=(response_function_calling['function_call'])['name']
print(function_name)

list_latest_episode_of_spotify_show


In [ ]:
import json
show_name=eval(response_function_calling['function_call']['arguments'])['show_name']
print(show_name)
limit=eval(response_function_calling['function_call']['arguments'])['limit']
print(limit)

Globant
10


In [ ]:
arguments = json.loads(response_function_calling["function_call"]["arguments"])
arguments

{'show_name': 'Globant', 'limit': 10}

In [ ]:
available_functions = {
    'list_latest_episode_of_spotify_show': list_latest_episode_of_spotify_show
}

In [ ]:
function_response = available_functions[function_name](
    show_name=arguments.get("show_name"),
    limit=arguments.get("limit"),
)
pprint.pprint(function_response, width=150)

('[{"release_date": "2023-06-26", "episode_name": "Qu\\u00e9 es el Learning Match - Santos Videla (Globant)", "url": '
 '"https://open.spotify.com/episode/568pXcS72Cg9R8RlLsQCzP"}, {"release_date": "2023-06-20", "episode_name": "Argentinian tech giant Globant plans '
 'to double down on Europe: A chat with Fernando Matzkin, who\'s leading the charge", "url": '
 '"https://open.spotify.com/episode/2e7tBHgZhlsnekzoCf1xus"}, {"release_date": "2023-06-13", "episode_name": "Entrevista a Andrey Luj\\u00e1n, '
 'Managing Director de Globant en Chile", "url": "https://open.spotify.com/episode/1lYESmQLnB4PABIqH4xJlO"}, {"release_date": "2023-02-17", '
 '"episode_name": "C\\u00f3mo sigue la deuda en pesos, cu\\u00e1ndo llegan los billetes de $2.000 y Globant rompe r\\u00e9cords", "url": '
 '"https://open.spotify.com/episode/3jusDSalZOXIpT29jmlaPS"}, {"release_date": "2023-01-13", "episode_name": "#4 Elena Morettini, Global Head '
 'Sustainable Business en Globant", "url": "https://open.spotify.co

In [ ]:
import pprint
result_data = list_latest_episode_of_spotify_show('Google', limit=50)
result_list = json.loads(result_data)
pprint.pprint([data['episode_name'] for data in result_list])

['The Ultimate Spy Tool for Google Ads: Get Ahead of Your Competitors Now',
 '🟢 Consultorio de Bolsa | ¿Oro físico? ¿Google? ¿Aerolíneas?',
 '178. The Keyword Connection: Bridging Google Shopping Ads and SEO for '
 'Ultimate Success',
 'BONUS Episode! Automating Your Google Ads - Keywords & Bidding (Episode '
 '#365)',
 'Uso da IA para desenvolvedores',
 '#042 Google Ads für lokale Unternehmen',
 'Google se ríe de Apple, precio del Nothing Phone 2 y Peso Pluma | Ep25 S04',
 'Staying One Step Ahead: Expert Tips to Protect Your Business from Dirty '
 'Competition',
 'Google y su solución al problema de las imágenes generadas por IA',
 'Getting Started with Google Ads in your Shopify Store',
 '¿Qué opinan los que han probado el nuevo Google SGE? - Actualidad SEO #221',
 'Google’s Sundar Pichai Talks Seach, AI, and Dancing with Microsoft',
 '#041 Augen auf bei der Google Ads Agenturwahl',
 'Nacer en GOOGLE, pasar por WAZE y fundar VIVLA: La Historia de Carlos Gómez '
 '- Podcast #289',
 '#

In [ ]:
%%writefile spotify_utils.py
import json
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

client_credentials_manager = SpotifyClientCredentials()
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

def list_latest_episode_of_spotify_show(show_name, limit=20):
    """Get the latest episodes of a determine show or episode in a given name"""
    result_fetch = sp.search(show_name, limit=40, offset=0, type='episode',market='ES')
    final_list = []
    for item in result_fetch['episodes']['items']:
        if show_name.lower() in item['description'].lower():
            final_list.append({'release_date':item['release_date'],'episode_name':item['name'],'url':item['external_urls']['spotify']})
    final_list = sorted(final_list, key=lambda x: x['release_date'], reverse=True)
    return json.dumps(final_list[:limit])

Writing spotify_utils.py


In [ ]:
import spotify_utils
function_object = getattr(spotify_utils, function_name)
function_response = function_object(**arguments)
pprint.pprint(function_response, width=150)

('[{"release_date": "2023-06-26", "episode_name": "Qu\\u00e9 es el Learning Match - Santos Videla (Globant)", "url": '
 '"https://open.spotify.com/episode/568pXcS72Cg9R8RlLsQCzP"}, {"release_date": "2023-06-20", "episode_name": "Argentinian tech giant Globant plans '
 'to double down on Europe: A chat with Fernando Matzkin, who\'s leading the charge", "url": '
 '"https://open.spotify.com/episode/2e7tBHgZhlsnekzoCf1xus"}, {"release_date": "2023-06-13", "episode_name": "Entrevista a Andrey Luj\\u00e1n, '
 'Managing Director de Globant en Chile", "url": "https://open.spotify.com/episode/1lYESmQLnB4PABIqH4xJlO"}, {"release_date": "2023-02-17", '
 '"episode_name": "C\\u00f3mo sigue la deuda en pesos, cu\\u00e1ndo llegan los billetes de $2.000 y Globant rompe r\\u00e9cords", "url": '
 '"https://open.spotify.com/episode/3jusDSalZOXIpT29jmlaPS"}, {"release_date": "2023-01-13", "episode_name": "#4 Elena Morettini, Global Head '
 'Sustainable Business en Globant", "url": "https://open.spotify.co

In [ ]:
def askToSpotifyBot(user_input, functions=functions):
    """
    Interacts with a Spotify bot to find podcasts and return a list with markdown links.

    Args:
        user_input (str): The user's input/query.
        functions (dict): A dictionary containing available functions for interaction.

    Returns:
        str: Formatted response containing podcast list or function output.
    """

    # Initial system message
    messages = [{'role': 'system', 'content': """
        You are an assistant helping me find podcasts on Spotify and returning the entire list.
        Please reply in markdown format and add the link at the end of each episode using [Listen here](url)""" }]

    # Add user input to messages
    messages.append({"role": "user", "content": user_input})

    # Call the GPT-3 model for initial completion
    completion = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=messages,
        functions=functions
    )

    response_function_calling = completion.choices[0].message

    if response_function_calling is not None and response_function_calling.get('function_call'):
        # If a function call is detected
        function_name = response_function_calling['function_call']['name']
        function_args = json.loads(response_function_calling["function_call"]["arguments"])
        function_response = available_functions[function_name](**function_args)

        # Call GPT-3 again with function response
        _response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo",
            messages=[
                {"role": "user", "content": user_input},
                response_function_calling,
                {
                    "role": "function",
                    "name": function_name,
                    "content": function_response,
                },
            ],
            functions=functions
        )

        return _response.choices[0]["message"]["content"].strip()

    else:
        # If no function call detected, return the initial completion
        return completion.choices[0]["message"]["content"].strip()


In [ ]:
from IPython.display import Markdown
Markdown(askToSpotifyBot("What are the latest 7 episodes of Twitter? Be concise"))

Here are the latest 7 episodes of Twitter:

1. "Campaigns Look To Court, France Seeks Calm, Twitter Rations Views" - [Listen Here](https://open.spotify.com/episode/2jsAw80HftkrlSaCa6KV1y)
2. "Peak social media: Trouble at Twitter" - [Listen Here](https://open.spotify.com/episode/12QgP7MQGE52vRsIUK6AqU)
3. "#387 – George Hotz: Tiny Corp, Twitter, AI Safety, Self-Driving, GPT, AGI & God" - [Listen Here](https://open.spotify.com/episode/0TA2uCZUQDF8AdF5dUofAi)
4. "#STR50 To Be Directed By...? | Twitter Time Pass Space" - [Listen Here](https://open.spotify.com/episode/5TBS88QL3C121WoSUM9ih7)
5. "Neues vom Twitter-Stresstest (mit Frederik von Castell)" - [Listen Here](https://open.spotify.com/episode/7ff4j9ccXCMDmFRO1v24AW)
6. "Tesla’s Rising Value, Meta’s AI, and Twitter’s Ad Problem" - [Listen Here](https://open.spotify.com/episode/7dfBNbULR4RBRFlpfzD49t)
7. "Mari Selvaraj Controversial Issue Is The Promotional Stunt For The Release Of Maamannan Movie? - Twitter Time Pass Space" - [Listen Here](https://open.spotify.com/episode/1vqzLETmQzsMimrwlh3xjf)

In [ ]:
Markdown(askToSpotifyBot("What are the latest 8 episodes of Globant? Group by year"))

The latest 8 episodes of Globant, grouped by year, are as follows:

2023:
- "Qué es el Learning Match - Santos Videla (Globant)" - Released on June 26th. [Listen here](https://open.spotify.com/episode/568pXcS72Cg9R8RlLsQCzP)
- "Argentinian tech giant Globant plans to double down on Europe: A chat with Fernando Matzkin, who's leading the charge" - Released on June 20th. [Listen here](https://open.spotify.com/episode/2e7tBHgZhlsnekzoCf1xus)
- "Entrevista a Andrey Luján, Managing Director de Globant en Chile" - Released on June 13th. [Listen here](https://open.spotify.com/episode/1lYESmQLnB4PABIqH4xJlO)

2022:
- "Cómo sigue la deuda en pesos, cuándo llegan los billetes de $2.000 y Globant rompe récords" - Released on February 17th. [Listen here](https://open.spotify.com/episode/3jusDSalZOXIpT29jmlaPS)
- "#4 Elena Morettini, Global Head Sustainable Business en Globant" - Released on January 13th. [Listen here](https://open.spotify.com/episode/6uzo9K9ydjWjlRuLU6AN8u)
- "Alerta por los activos de los bancos, cuánto crecerá el PBI en 2023 y adquisición de Globant" - Released on January 11th. [Listen here](https://open.spotify.com/episode/5ArNty2xDDuueBot8j6w6z)

2021:
- "Q&A con Nicolás Dujovne, los planes de Migoya para Globant y el ajuste de las jubilaciones en 2023" - Released on October 12th. [Listen here](https://open.spotify.com/episode/2HTdIPM0aSuO9JjkHnldzS)

2020:
- "Una alianza que potencia la experiencia de cliente, YPF y Globant." - Released on June 3rd. [Listen here](https://open.spotify.com/episode/47pXZNEZqkT4H6TiIOlrsT)

In [ ]:
Markdown(askToSpotifyBot("What are the latest 15 episodes of Bitcoin? Group by month and change 'Listen here' link to 'Escuchame'"))

Here are the latest 15 episodes of Bitcoin grouped by month:

- July 2023:
  - Hola, instituciones: Bitcoin viene a pegar fuerte - [Escuchame](https://open.spotify.com/episode/329uVB6Zl3MsGKDHQCZNUz)

- June 2023:
  - 817 Empresas millonarias ¿Pueden acaparar las compras de bitcoin? - [Escuchame](https://open.spotify.com/episode/4Juq0UwLbVy3rlXOQI4XZV)
  - L196 - El adiós a España de la Familia Bitcoin - [Escuchame](https://open.spotify.com/episode/0m03EWBBXa2iWy9u0QrLs2)
  - ¿Qué hay que saber sobre la propuesta de ETF Bitcoin de Blackrock? - [Escuchame](https://open.spotify.com/episode/0uBCAOgXROI4bcO7eql2p0)
  - BBVA y el bitcoin, euríbor, Endesa, Iberdrola e Indra - [Escuchame](https://open.spotify.com/episode/06gemnHN4YKdKs1OvrRDkm)
  - 813 Dispositivos DIY para... ¿MINAR BITCOIN? - [Escuchame](https://open.spotify.com/episode/4R1kNQsL0C9OnUptEj08io)
  - Yago Álvarez Barba: El momento en que Bitcoin perdió su potencial - [Escuchame](https://open.spotify.com/episode/4J9JxABzXWZnGXTP8DUF6k)
  - Episode 163 - Bitcoin, ETF & Regulation - Chancen und Risiken - [Escuchame](https://open.spotify.com/episode/1EBBqjGsQpgn1kT00DkHo0)
  - BITCOIN: SALIDA DE EMERGENCIA - Vlog de Marc Vidal - [Escuchame](https://open.spotify.com/episode/3mP3f2D8wmjt2pTRlS4rN6)
  - L195 - Bitcoin es peor es mejor por Gwern Branwen - [Escuchame](https://open.spotify.com/episode/3XnuubEHXcgl2yHVx84jeI)
  - S05E24 - Mister Bitcoin Standard - [Escuchame](https://open.spotify.com/episode/7LiX8sXq3MNBCnPawxQopD)
  - El culto a Bitcoin - [Escuchame](https://open.spotify.com/episode/0ICfefzUbWHZrZRx2AYNRF)
  - 811 Binance aceptará lightning network - [Escuchame](https://open.spotify.com/episode/5DRUAozUxTTvr6SYGvGYgO)
  - Manuel Beaudroit: Por qué estar en Bitcoin - [Escuchame](https://open.spotify.com/episode/0BZX6aRQYbWB0HSi7v6BDt)
  - Episode 162 - Cointracking: Bitcoin-Steuern dokumentieren & Steuerreports erstellen mit Moritz Nold - [Escuchame](https://open.spotify.com/episode/7h62pfQVRwWQy3XHRgh94i)